In [ ]:
import pandas as pd

In [ ]:
%run ../modules/utils.ipynb
%run ../modules/cds.ipynb
%run ../modules/preprocessing.ipynb

In [ ]:
df_mix5_s1 = load_excel('/Users/xyuan/Documents/SeqDataSets/Ning1024/5mix_MFE.xlsx', '3-biotin')
df_mix5_s2 = load_excel('/Users/xyuan/Documents/SeqDataSets/Ning1024/5mix_MFE.xlsx', '5-sulfo-Cy3')
df_mix5_s1.shape, df_mix5_s2.shape

In [ ]:
fpath = "/Users/xyuan/Documents/SeqDataSets/mix RNA MFE/13 MIX reading/12 mixures of 3-biotin/12 mixtures_3'-Biotin_MFE.xls"
# fpath = "/Users/xyuan/Documents/SeqDataSets/Ning1024/12mixtures_3'-Biotin_MFE.xls"
df_mix12_s1 = load_excel(fpath, '12MIX')
fpath = "/Users/xyuan/Documents/SeqDataSets/mix RNA MFE/13 MIX reading/12 mixtures of 5-sulfo-Cy3/12 mixtures (5-sulfo-Cy3)_MFE.xlsx"
df_mix12_s2 = load_excel(fpath, '12mix')
df_mix12_s1.shape, df_mix12_s2.shape

In [ ]:
plotly_zone(df_mix12_s1, y='Vol')

In [ ]:
# df_mix5_s1 = df_mix5_s1[(df_mix5_s1.RT<11)&(df_mix5_s1.Mass<1E4)].copy()
# df_mix5_s2 = df_mix5_s2[(df_mix5_s2.RT<15)&(df_mix5_s2.Mass<1E4)].copy()
# plotly_zone(df_mix5_s1)
# plotly_zone(df_mix5_s2)

df_mix12_s1 = df_mix12_s1[(df_mix12_s1.RT<15)].copy()
df_mix12_s2 = df_mix12_s2[(df_mix12_s2.RT<15)].copy()
plotly_zone(df_mix12_s1)
plotly_zone(df_mix12_s2)

In [ ]:
dfm = match_dfs(df_mix5_s1, df_mix12_s1)
plotly_zone(dfm)
dfm.shape

In [ ]:
# df_biotin3p = match_dfs(df_mix5_s2, df_mix5_s1, shift=694.2397+61.9557)
# df_cy35p = match_dfs(df_mix5_s1, df_mix5_s2, shift=852.1934-H2O)

df_biotin3p = match_dfs(df_mix12_s2, df_mix12_s1, shift=694.2397+61.9557)
df_cy35p = match_dfs(df_mix12_s1, df_mix12_s2, shift=852.1934-H2O)

In [ ]:
plotly_zone(df_biotin3p)
plotly_zone(df_cy35p)
df_biotin3p.shape, df_cy35p.shape

In [ ]:
# bio3_fullmass = 6954.9478 #7088.0479 #7522.068 #7079.051 # # #6781.0426 #  # # #
bio3_fullmass = 6353.872 #10283.47 #7073.039
shift = bio3_fullmass + 852.1934 - H2O
cds = computational_data_seperation(df_biotin3p, df_cy35p, full_mass=shift)
plotly_zones(*cds, names=['SulfoCy3 5´', 'Biotin 3´'])

df_merge_5p3p = pd.concat([cds[0], cds[1]]).drop_duplicates()
bcr = base_calling_random(df_merge_5p3p)
plotly_basecalling(df_merge_5p3p, bcr[1])

In [ ]:
trna_path = '/Users/xyuan/Documents/SeqDataSets/tRNA/191126_sample analysis/191126s11_1000.xls'
df_trna = load_excel(trna_path)
df_trna.shape

In [ ]:
dfm = match_dfs(df_trna, df_trna, shift=694.2397)
plotly_zone(dfm)

In [ ]:
import multiprocessing
from collections import namedtuple
ShiftHit = namedtuple('ShiftHit', 'shift hit')

def func(df_sample, shift):
    dfm = match_dfs(df_sample, df_sample, shift=shift)
    sh = ShiftHit(shift=shift, hit=dfm.shape[0])
    return sh

PROCESSES = 16
params = [(df_trna, shift) for shift in np.arange(1, 100, 1)]
# params = [(df_sample, shift) for shift in np.arange(1, 64, 0.1)]
with multiprocessing.Pool(PROCESSES) as pool:
    shift_hits = pool.starmap(func, params)

df_shift_hits = pd.DataFrame(shift_hits)    